# 1. Binary search tree

A binary search tree is a binary tree T with each position p storing a key-value pair (k,v) such that:
    * Key stored in the left subtree of p is lesss than k
    * Key stored in the right subtree of p is greater than k


Proposition: an inorder traversal of a binary search tree visits positions in increasing order of their keys.
This proposition can proved by induction on the size of tree.

In generic binary tree is defined as a positional structure, allowing direct navigation using methods such as parent(p), left(p), right(p). With a binary search tree, we can provide additional navigation based on the natural order of the keys stored in the tree.
    * first()
    * last()
    * before(p)
    * after(p)

In [1]:

class TreeMap(LinkedBinaryTree, MapBase):
    
    class Position(LinkedBinaryTree.Position):
        def key(self):
            return self.element()._key
        
        def value(self):
            return self.element()._value
        
    def _subtree_search(self, p, k):
        """Return position of p's subtree having key k or last node searched"""
        if k == p.key():
            return p
        elif k < p.key():
            if self.left(p) is not None:
                return self._subtree_search(self.left(p), k)
        else:
            if self.right(p) is not None:
                return self._subtree_search(self.right(p), k)        
        return p
    
    def _subtree_first_position(self, p):
        """Return Position of first item in subtree rooted at p. This is the left most node in subtree rooted p"""
        walk = p
        while self.left(walk) is not None:
            walk = self.left(walk)
        return walk
    
    def _subtree_last_position(self, p):
        """Return position of last item in subtree rooted at p. This is the rightmost node in the tree rooted at p"""
        walk = self.right(p)
        while self.right(walk) is not None:
            walk = self.right(p)
        return walk
    
    def first(self):
        """Return the first position in the tree (or None if empty)"""
        if len(self) > 0 :
            return self._subtree_first_position(self.root())
        else:
            return None
    
    def last(self):
        """Return the last position in the tree (or None if empty)"""
        if len(self) > 0:
            return self._subtree_last_position(self.root())
        else:
            return None
    
    def before(self, p):
        """Return the Position just before p in the natural order
        Return None if p is the first position
        """
        self._validate(p)
        if self.left(p) is not None:
            return self._subtree_last_position(self.left(p))
        else:
            walk = p
            above = self.parent(p)
            while above is not None and walk == self.left(above):
                walk = above
                above = self.parent(walk)
            return above
        
    def after(self, p):
        """Return the position just after p in the natural order
        Return None if p is the last positon"""
        self._validate(p)
        if self.right(p) is not None:
            return self._subtree_first_position(self.right(p))
        else:
            walk = p
            above = self.parent(p)
            while above is not None and walk == self.right(above):
                walk = above
                above = self.parent(walk)
            return above
            
    def find_position(self, k):
        """Return position with key k or else neighbor (or None if empty)"""
        if self.is_empty():
            return None
        else:
            p = self._subtree_search(self.root(), k)
            self._rebalance_access(p)
            return p
        
    def find_min(self):
        """Return key-value pair with minimum key or None if empty"""
        if self.is_empty():
            return None
        else:
            p = self.first()
            return p.key(), p.value()
    
    def find_ge(self, k):
        """Return key-value pair with least key greater than or equal to k
        Return None if there does not exist such a key
        """
        if self.is_empty():
            return None
        else:
            p = self.find_postion(k)
            if p.key() < k:
                p = self.after(p)
            if p is not None:
                return p.key(), p.value()
            else:
                return None
    
    def find_range(self, start, stop):
        """Iterate all key-value pairs such that start <= key < stop
        If start is None, iteration begins with minimum key of map
        If stop is None, iteration continues through the maximum key of map"""
        if not self.is_empty():
            if start is None:
                p = self.first()
            else:
                p =self.find_position(start)
                if p.key() < start:
                    p = self.after(p)
                while p is not None and (stop is None or p.key() < stop):
                    yield p.key(), p.value()
                    p = self.after(p)
    
    def __getitem__(self, k):
        """Return value associated with key k (raise KeyError if not found)"""
        if self.is_empty():
            raise KeyError('KeyError ' + repr(k))
        else:
            p = self._subtree_search(self.first(), k)
            self._rebalance_search()
            if k != p.key():
                raise KeyError('KeyError' + repr(k))
            return p.value()
        
    def __setitem__(self, k, v):
        """Assign value v to key k, overwriting existing value if present"""
        if self.is_empty():
            leaf = self._add_root(self._Item(k, v))
        else:
            p = self._subtree_search(self.root(), k)
            if p.key() == k:
                p.element()._value = v
                self._rebalance_search()
                return
            else:
                item = self._Item(k, v)
                if p.key() < k:
                    leaf = self._add_right(p, item)   # inherited from LinkedBinaryTree
                else:
                    leaf = self._add_left(p, item)
        self._rebalance_insert(leaf)
        
    def __iter__(self):
        """Generate an iteration of all keys in the map in order"""
        p = self.first()
        while p is not None:
            yield p.key()
            p = self.after(p)
            
    def delete(self, p):
        """Remote the item at given position"""
        self._validate(p)
        if self.left(p) and self.right(p):
            replacement = self._subtree_last_position(self.left(p))
            self._replace(p, replacement.element())    # from linkedBinaryTree
            p = replacement
        parent = self.parent(p)
        self._delete(p)
        self._rebalance_delete(parent)
        
    def __delitem__(self, k):
        """Remove item associated with key k"""
        if not self.is_empty():
            p = self._subtree_search(self.root(), k)
            if k == p.key():
                self.delete(p)
                return
            self._rebalance_access(p)
        raise KeyError('KeyError: ' + k)
        
    # The following 3 methods are used to rebalance tree such as AVL or Red-black tree
    def _rebalance_insert(self, p):
        pass
    
    def _rebalance_delete(self, p):
        pass
    
    def _rebalance_access(self, p):
        pass      
                
    # The following methods implemnt single rotation and trinode restructure.
    def _relink(self, parent, child, make_left_child):
        """Relink parent node with child node"""
        if make_left_child:
            parent._left = child
        else:
            parent._right = child
        if child is not None:
            child.parent = parent
    
    def _rotate(self, p):
        """Rotate position p aboves its parent"""
        x = p.node
        y = x._parent
        z = y._parent
        if z is None:
            self._root = x                  # x become root
            x._parent = None
        else:
            self._relink(z, x, y == z._left) # x becomes direct child of z
        if x == y._left:
            self._relink(y, x._right, True)  # x._right becomes left child of y
            self._relink(x, y, False)        # y become right child of x
        else:
            self._relink(y, x._left, False)  # x._left becomes right child of y
            self._relink(x, y, True)         # y becoms left child of x
    
    def _restructure(self, x):
        """Perform trinode restructure of Position x with parent/grandparent"""
        y = self.parent(x)
        z = self.parent(y)
        if (x == self.right(y)) == (y == self.right(z)):
            self._rotate(y)                  # single rotation of y
            return y
        else:
            self._rotate(x)
            self._rotate(x)
            return x
            
    

NameError: name 'LinkedBinaryTree' is not defined

## 1.1 Performance of Binary Search Tree
|Operation|Running Time|
|----------|-----------
|k in T|O(h)|
|T[k], T[k] = v| O(h)|
|T.delete(p), del T[k]| O(h)|
|T.find_position(p)|O(h)|
|T.first(), T.last(), T.find_min(), T.find_max()| O(h)|
|T.before(), T.after()|O(h)|
|T.find_lt(k), T.find_le(k), T.find_gt(k), T.find_ge(k)|O(h)|
|T.find_range(start, stop)|O(h+s)|
|iter(T), reverse(T)|O(n)|

# 2. Balance Search Tree

In the closing of the previous section, we noted that if we could assume a random series of insertions and removals, the standard binary search tree supports O(logn) expected running times for the basic map operations. However, we may only claim
O(n) worst-case time, because some sequences of operations may lead to an unbalanced tree with height proportional to n.

In the remainder of this chapter, we explore four search tree algorithms that provide stronger performance guarantees. Three of the four data structures (AVL trees, splay trees, and red-black trees) are based on augmenting a standard binary search tree with occasional operations to reshape the tree and reduce its height

All Balance Trees above based on the restructure algorithm:
    Algorithm restructure(x):
        input: a position x of a binary search tree T that has both a parent y and grand parent z
        output: tree T after a trinode reconstructing involving positions x,y and z.
        
        1. let a,b,c be the left-to-right listing of the positions x,y,z. and T1, T2, T3, T4 be a left-to-right listing of the four subtree of x, y, z not rooted at x, y, z
        2. replace the subtree rooted at z with a new subtree rooted at b
        3. let a be the left child of b and let T1, T2 be the left and right subtrees of a.
        4. let c be the right child of b and let T3, T4 be the left and right subtrees of c

## 2.1 Hooks for Rebalancing Operations

Our implementation of the basic map operations in previous includes strategic calls to three nonpublic methods that serve as hooks for rebalancing algorithms:

    • A call to rebalance insert(p) is made from within the setitem method immediately after a new node is added to the tree at position p.
    • A call to rebalance delete(p) is made each time a node has been deleted from the tree, with position p identifying the parent of the node that has just been removed. Formally, this hook is called from within the public delete(p) method, which is indirectly invoked by the public delitem (k) behavior.
    • We also provide a hook, rebalance access(p), that is called when an item at position p of a tree is accessed through a public method such as getitem. This hook is used by the splay tree structure (see Section 11.4) to restructure a tree so that more frequently accessed items are brought closer to the root.

## 2.2 Non public method for rotating and restructuring

A second form of support for balanced search trees is our inclusion of non public methods _rotate and _reconsture, implement a single rotation and trinode restructuring.

# 3. AVL Tree

AVL tree is a binary search tree has Height-Balance Property: for every position p of T, the heights of the children of p differ by at most 1.

<b>Proposition</b>: The height of an AVL tree storing n entries is O(logn)

The insertion and deletion operations for AVL trees begin similarly to the corresponding operations for standard binary search tree, but with post-processing for each operation to restore the balance of any portions of the tree.

## 3.1 Insertion 
    The insertion of a new item in a BST, results in a new node at a leaf postion p. This action may violate the height-balance property yet the only positions that may become unbalanced are ancestors of p. 
    We restore the balance property of AVL by a simple "search-and-repair" strategy. In particular, let z be the first position we encounter in going up from p toward root of tree T such that z is unbalanced. Also let y denote the child of z with higher height (and note that y must be the ancestor of p). Finally, let x is the child of y with higher height (and note that x must be the ancestor of p or p itself). We rebalance subtree rooted at z by calling the trinode restructuring method, restructuring(x). 
    The subtree rooted at z after restructuring is balance and all parent of this subtree also balanced. This is heigh-balance property <b>globally</b>.
    
## 3.2 Deletion
    The deletion of a item in a BST, results may be violate the height-balance property in an AVL tree. If p is the position of parent of removed node, there maybe an unbalanced node in the path from p to root of AVL tree. As with insertion, We restore the balance property of AVL by a simple procedure: let z be the first position unbalanced, y be the child of z with higher height (note that y must be not the ancestor of p), x is the child of y which higher heigh or child of y on the same side as y. In any case, we then perform a restructuring(x).
    The restructured subtree has locally property. Maybe after restructured, the parent become unbalanced. So, after rebalancing z, we continue upward T looking for unbalanced positions and continue restructuring.
    

## 3.3 Performance of AVL tree

    The complexity of AVL tree compare to BST change from log(h) to logn.
|Operation|Running Time|
|----------|-----------
|k in T|O(logn)|
|T[k], T[k] = v| O(logn)|
|T.delete(p), del T[k]| O(logn)|
|T.find_position(p)|O(logn)|
|T.first(), T.last(), T.find_min(), T.find_max()| O(logn)|
|T.before(), T.after()|O(logn)|
|T.find_lt(k), T.find_le(k), T.find_gt(k), T.find_ge(k)|O(logn)|
|T.find_range(start, stop)|O(logn+s)|
|iter(T), reverse(T)|O(n)|

In [1]:
class AVLTree(TreeMap):
    """Sorted map implementation using an AVL tree"""
    
    #---------------- Nested Node class ------------------
    class _Node(TreeMap._Node):
        __slots__ = '_height'
        
        def __init__(self, element, parent = None, left = None, right = None):
            super.__init__(element, parent, left, right)
            self._height = 0    # will be computed during balancing
            
        def left_height(self):
            return self._left._height if self._left is not None else 0
        
        def right_height(self):
            return self._right._height if self._right is not None else 0
        
        
    def _recompute_height(self, p):
        p._node._height = 1 + max(p._node.left_height(), p._node.right_height())
        
    def _isbalanced(self, p):
        return abs(p._node.left_height() - p._node.right_height()) <= 1
    
    def _tall_child(self, p, favorleft = False):
        if p._node.left_height() + (1 if favorleft else 0) > p._node.right_height():
            return self.left(p)
        else:
            return self.right(p)
        
    def _tall_grandchild(self, p):
        child = self._tall_child(p)
        # if child is on left, favor left granchild; else favor right grandchild
        alignment = (child == self.left(p))
        return self._tall_child(child, alignment)
    
    def _rebalance(self, p):
        while p is not None:
            old_height = p._node._height
            if not self._isbalance(p):
                p = self._restructure(self._tall_grandchild(p))
                self._recompute_height(self.left(p))
                self._recompute_height(self.right(p))
                
            self._recompute_height(p)
            if p._node._height == old_height:
                p = None
            else:
                p = self.parent(p)
                
    # Overwrite 
    def _rebalance_insert(self,p):
        return self._rebalance(p)
    
    def _rebalance_delete(self, p):
        return self._rebalance(p)
                

NameError: name 'TreeMap' is not defined

# 4. Splay Tree

Splay tree is conceptually quite different from the other BST. Splay tree does not strictly enforce a logarithm upper bound on the height of tree, and no addtional height, balance, color,... data associated with the nodes of the tree.

The splay tree based on splay operation. The operation causes more frequently accessed elements to remain nearer to the root, thereby reducing search times. The suprising things about splaying is that it allows us to guarantee a logarithmic amortized running time, for insertions, deletions, and searches.


## 4.1 Splaying

Given a node x of BST tree T, we splay x by moving x to the root of T through a sequence of restructurings. The specific operation we perform to move x depends upon the relative positions of x, its parent y and its grandparent z. There are 3 cases:
    * zig-zig: the node x and y are both left of its parent. We promote x, making y a child of x, and z a child of y while maintain the inorder relationships on the nodes in T.
    * zig-zag: one of x and y is a left child of its parent and the other is right child.We promote x by making x have y and z as it children, while maintain the inorder relationships on the nodes of T.
    * zig: x does not have grandparent. We promote x by making y a child of x and maintain the inorder relationships on the nodes of T.

In [ ]:
class SplayTreeMap(TreeMap):
    """Sorted map implementation using a splay tree"""
    def _splay(self, p):
        while p != self.root():
            parent = self.parent(p)
            grand = self.parent(parent)
            
            if grand is None:
                self._rotate(p)
            elif (parent == self.left(grand) == (p == self.left(parent))):
                self._rotate(parent)
                self._rotate(p)
            else:
                self._rotate(p)
                self._rotate(p)
                
    def _rebalance_insert(self, p):
        if p is not None:
            self._splay(p)
    
    def _rebalance_delete(self, p):
        if p is not None:
            self._replay(p)

## 4.2 Amortized analysis of splaying

# 5. Red-Black Tree

Although AVL trees and (2,4) trees have a number of nice properties, they also have some disadvantages. For instance, AVL trees may require many restructure operations (rotations) to be performed after a deletion, and (2,4) trees may require many split or fusing operations to be performed after an insertion or removal. The data structure we discuss in this section, the red-black tree, does not have these drawbacks; it uses O(1) structural changes after an update in order to stay balanced.

Formally, a red-black tree is a binary search tree with nodes colored red and black in a way that satisfies the following properties:

<b>Root Property</b>: The root is black.

<b>Red Property</b>: The children of a red node (if any) are black.

<b>Depth Property</b>: All nodes with zero or one children have the same black depth, defined as the number of black ancestors. (Recall that a node is its own ancestor)

## 5.1 Insertion

The algorithm insertion of a pair key-value (k,v) into a red-black tree T initially proceeds as in the standard binary search tree. Namely, we search for key k in T until we reach a null subtree, and we introduce a new leaf x at that position, storing the item. In a special case, x is root node of tree then x has black color. In all other case, we color x red. This insertion preserves the root and depth properties of tree T but may violate Red property. In deed, if x is not root of T, y is parent of x is red. Note that by the Root property, y cannot be the root of T, and by the red property the parent z of y is black. Since x and its parent y are red but x's grandparent z is black, we call this violation of the RED property a <b>double red</b> at node x.

### Case 1: The sibling s of y is Black or None
Fix this by performming a trinode restructuring: restructure(x)
After restructure(x) operation, we color b black and a,c red. 
Note that the portion of any path through the restructured part of the tree is incident to exactly one black node, both before and after the trinode restructuring.

### Case 2: The sibling s of y is red
To fix this we do a coloring: we color y and s black and their parent z red (unless z is root, z is black). Notice that, unless z is root, the portion path through the affected part of the tree is incident to exactly one black node, both before and after coloring. Thereforce, the black depth of the tree is unaffected by the recoloring unless z is the root, in which case it is increased by one.
However, it is possible that the double-red problem reappears after such a recoloring, albeit higher up in the tree T, since z may have a red parent. In that case, we continue coloring uppper until we finally resolve the double-red problem.

## 5.2 Deletion

Deleting an item with key k from the red-black tree T initially proceeds as for a binary search tree. Structurally, the process result in the removal a node that has at most one child and the promotion of its remaining child.

If the removed node was red, this structural change does not affect the black depths of any paths in the tree, nor introduce any red violation and so the resulting tree remains a valid red-black tree. 

If the removed node was black, then it either had zero children or it had one child that was the red leaf. In the latter case, the removed node represents the black part of a corresponding 3-node and we restore the red-property by recoloring the promoted child to black. 

The more complex case is when a (nonroot) black leaf is removed. To remedy this scenario, we consider 3 case:
### Case 1: Node y is black and has a red child x
### Case 2: Node y is black and both children of y are black or None
### Case 3: Node y is red

In [ ]:
class RedBlackTree(TreeMap):
    """Sorted map implementation using a red-black tree"""
    class _Node(TreeMap._Node):
        """Node class for red-black tree maintains bit that denotes color"""
        __slots__ = '_red'
        
        def __init__(self, element, parent = None, left = None, right = None):
            super().__init__(element, parent, left, right)
            self._red = True
            
    # ------------------ Positional-based utility methods---------------------------#
    def _set_red(self, p): 
        p._node._red = True
    
    def _set_black(self, p):
        p._node._red = False
        
    def _set_color(self, make_red):
        p._node._red = make_red
        
    def _is_red(self, p):
        return p is not None and p._node._red
    
    def _is_red_leaf(self, p):
        return self._is_red(p) and self.is_leaf(p)
    
    def _get_red_child(self, p):
        """Return the red child of p (or None if no such child)"""
        for child in (self.left(p), self.right(p)):
            if self._is_red(child):
                return child;
        return None
    
    #------------------ support for insertion ---------------------------------#
    def _rebalance_insert(self, p):
        self._resolve_red(p)
        
    def _resolve_red(self, p):
        if self.is_root(p):
            self._set_black(p)
        else:
            parent = self.parent(p)
            if self._is_red(parent):                # double red problem
                uncle = self.sibling(parent)
                if not self._is_red:                # case 1
                    middle = self._restructure(p)
                    self._set_black(middle)
                    self._set_red(self.left(middle))
                    self._set_red(self.right(middle))
                else:                               # case 2
                    grand = self.parent(parent)
                    self._set_red(grand)
                    self._set_black(self.left(grand))
                    self._set_black(self.right(grand))
                    self._resolve_red(grand)
    
    # ---------------- support for deletion -----------------------------------#
    def _rebalance_delete(self, p):
        if len(self) == 1:
            self._set_black(self.root())
            elif p is not None:
                n = self.num_children(p)
                if n == 1:
                    c = next(self.children(p))
                    if not self._is_red_leaf(c):
                        self._fix_deficit(p, c)
                elif n == 2:
                    if self._is_red_leaf(self.left(p)):
                        self._set_black(self.left(p))
                    else:
                        self._set_black(self.right(p))
                        
    def _fix_deficit(self, z, y):
        """Resolve black deficit at z where y is root of z's heavier subtree"""
        if not self._is_red(y):        # y is black 
            x = self._get_red_child(y)
            if x is not None:          # case 1: y is black and has red child x, do "transfer"
                old_color = self._is_red(z)
                middle = self._restructure(x)
                self._set_color(middle, old_color)     # middle get old color of z
                self._set_black(self.left(midlle))     # children become black
                self._set_black(self.right(middle))
            else:                   # case 2: y is black but no red children; do "fusion"
                self._set_red(y)
                if self._is_red(z):
                    self._set_black(z)
                elif not self.is_root(z):
                    self._fix_deficit(self.parent(z), self.sibling(z))
        else:                        # case 3: y is red; rotate
            self._rotate(y)
            self._set_black(y)
            self._set_red(z)
            if z == self.right(y):
                self._fix_deficit(z, self.left(z))
            else:
                self._fix-deficit(z, self.right(z))                